In [ ]:
from pyannote.audio import Pipeline

# Replace 'YOUR_AUTH_TOKEN' with your Hugging Face access token
auth_token = "YOUR_AUTH_TOKEN"

# Load the pre-trained pyannote.audio pipeline for speech activity detection
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=auth_token)

# Path to the audio file
audio_file = "../data/speaker_diarization/speech_mix_1.wav"

# Apply the pipeline to the audio file
segmentation = pipeline(audio_file)

# Print speech vs non-speech timestamps
print("Speech vs Non-Speech Timestamps:")
for segment, track, label in segmentation.itertracks(yield_label=True):
    print(f"Start: {segment.start:.2f}s, End: {segment.end:.2f}s, Label: {label}")